In [ ]:
# Problem 2

# Part 1: 3 points

# Load the data from the file data/websites.csv and estimate the transition matrix of the Markov chain
# Store the estimated transition matrix in the variable problem2_transition_matrix below
# problem2_transition_matrix = XXX # A numpy array of shape (problem2_n_states, problem2_n_states)
# Store the number of states in the variable problem2_n_states below
# problem2_n_states = XXX # An integer
# ==================================== #
import numpy as np
import pandas as pd

# Load data
df = pd.read_csv('data/websites.csv')

# Find the number of states (pages)
pages = np.union1d(df['source'].unique(), df['destination'].unique())
problem2_n_states = int(pages.max() + 1)  

# Initialize transition count matrix
transition_counts = np.zeros((problem2_n_states, problem2_n_states), dtype=int)

# Fill transition counts
for _, row in df.iterrows():
    transition_counts[int(row['source']), int(row['destination'])] += 1

# Normalize to get the transition probability matrix
row_sums = transition_counts.sum(axis=1, keepdims=True)
# Avoid division by zero
row_sums[row_sums == 0] = 1
problem2_transition_matrix = transition_counts / row_sums

# Part 2: 4 points

# Simulate the website load times for the next page of 10000 users that are currently on page 8 (recall indexing starts at 0) when we only preload the most likely page.
# Store the simulated page load times in the variable problem2_page_load_times_top below
# problem2_page_load_times_top = XXX # A numpy array of shape (10000,)

# Repeat the simulation of load times for the next page of 10000 users that are currently on page 8 when we preload the two most likely pages.
# Store the simulated page load times in the variable problem2_page_load_times_two below
# problem2_page_load_times_two = XXX # A numpy array of shape (10000,)

# ===================================== #
# For reproducibility

np.random.seed(42)

n_sim = 10000
start_page = 8

# Probabilities of transitions from page 8
probs_from_8 = problem2_transition_matrix[start_page]

# Draw the next page for each user
next_pages = np.random.choice(problem2_n_states, size=n_sim, p=probs_from_8)

# Preload only the most likely next site
most_likely = np.argmax(probs_from_8)
problem2_page_load_times_top = np.where(
    next_pages == most_likely,
    np.random.exponential(1/20, n_sim),
    np.random.exponential(1/3, n_sim)
)

# Preload the two most likely pages
top_two = np.argsort(probs_from_8)[-2:]
problem2_page_load_times_two = np.where(
    np.isin(next_pages, top_two),
    np.random.exponential(1/20, n_sim),
    np.random.exponential(1/3, n_sim)
)

# Part 3: 3 points

# Calculate the true expected load time for loading a page without pre-loading the next page and store it in the variable below
#problem2_avg = XXX # A float

# Is the average load time for loading a page without pre-loading the next page larger than the average load time for loading a page after pre-loading the next most likely page?
# problem2_comparison = XXX # True / False

# =================================  #

# Theoretical expected load time without pre-loading
problem2_avg = 1/3  # mean of Exp(3)

# Is the average load time with no pre-load larger than with preloading the top page?
problem2_comparison = problem2_avg > problem2_page_load_times_top.mean()


# Part 4: 4 points

# Begin by calculating the stationary distribution of the Markov chain and store it in the variable below
# WARNING: Since the transition matrix is not symmetric, numpy might make the output of the eigenvectors complex, you can use np.real() to get the real part of the eigenvectors
# Store the stationary distribution in the variable below called problem2_stationary_distribution
# problem2_stationary_distribution = XXX # A numpy array of shape (problem2_n_states,)

# Now use the above stationary distribution to calculate the average load time for loading a page after pre-loading the next most likely page according to the stationary distribution
# Store the average load time in the variable below
# problem2_avg_stationary = XXX # A float

# ===============================  #

# Calculate stationary distribution (left eigenvector of eigenvalue 1)
eigvals, eigvecs = np.linalg.eig(problem2_transition_matrix.T)
stat_idx = np.isclose(eigvals, 1)
stationary = np.real(eigvecs[:, stat_idx][:, 0])
problem2_stationary_distribution = stationary / stationary.sum()

# For each state, find the most probable next page
most_likely_next = np.argmax(problem2_transition_matrix, axis=1)

# Expected load time when preloading the most likely page from each state
expected_load_time_by_state = (
    problem2_transition_matrix[np.arange(problem2_n_states), most_likely_next] * (1/20) +
    (1 - problem2_transition_matrix[np.arange(problem2_n_states), most_likely_next]) * (1/3)
)
problem2_avg_stationary = np.dot(problem2_stationary_distribution, expected_load_time_by_state)




## Free text answer

Put the explanation for **part 3** of how you made the decision about `problem2_comparison` below this line in this **cell**. In order to enter edit mode you can doubleclick this cell or select it and press enter.
The decision for `problem2_comparison` is based on comparing the theoretical average load time for loading a page without pre-loading (which is the mean of an Exp(3) distribution, i.e., 1/3 seconds) to the empirical average load time obtained from the simulation where we preload the most likely next page (`problem2_page_load_times_top`). If the empirical average load time with preloading is **lower** than the theoretical average load time without preloading, then `problem2_comparison` is set to `True`, indicating that preloading the most likely page improves the average load time. Otherwise, it is `False`. This comparison is made directly by evaluating `problem2_avg > problem2_page_load_times_top.mean()`.


In [ ]:
# Problem 1


# Part 1

# def problem1_rejection(n_samples=1):
    # Distribution from part 1
    # write the code in this function to produce samples from the distribution in the assignment
    # Make sure you choose a good sampling distribution to avoid unnecessary rejections

    # Return a numpy array of length n_samples
    # return XXX

# ==========================  #
import numpy as np

def problem1_rejection(n_samples=1):
    samples = []
    M = 2 * (np.exp(1) - 1) / (np.exp(1) - 2)
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1)
        u = np.random.uniform(0, 1)
        fx = 2 * x * (np.exp(x**2) - 1) / (np.exp(1) - 2)
        if u < fx / M:
            samples.append(x)
    return np.array(samples)


# Part 2

# problem1_samples = XXX
# ==========================  #
import matplotlib.pyplot as plt

problem1_samples = problem1_rejection(100000)

# Plot histogram and true density
x_grid = np.linspace(0, 1, 200)
true_density = 2 * x_grid * (np.exp(x_grid**2) - 1) / (np.exp(1) - 2)

plt.hist(problem1_samples, bins=50, density=True, alpha=0.5, label='Samples')
plt.plot(x_grid, true_density, 'r-', label='True density')
plt.legend()
plt.xlabel('x')
plt.ylabel('density')
plt.title('Histogram of samples vs true density')
plt.show()


# Part 3

# problem1_integral = XXX
# ==========================  #
problem1_integral = np.mean(np.sin(problem1_samples))


# Part 4

# problem1_interval = [XXX,XXX]
# ==========================  #

n = len(problem1_samples)
a = 0
b = np.sin(1)
mean = problem1_integral
delta = np.sqrt(np.log(2/0.05) * (b - a) ** 2 / (2 * n))
problem1_interval = [mean - delta, mean + delta]



# Part 5

# def problem1_rejection_2(n_samples=1):
    # Distribution from part 2
    # write the code in this function to produce samples from the distribution in the assignment
    # Make sure you choose a good sampling distribution to avoid unnecessary rejections

    # Return a numpy array of length n_samples
    # return XXX

# ==========================  #

def problem1_rejection_2(n_samples=1):
    samples = []
    M = 21
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1/20)
        u = np.random.uniform(0, 1)
        fx = 20 * np.exp(20 - 1/x) * (1 + 1/x)
        gx = 20  # Uniform on [0, 1/20]
        if u < fx / (M * gx):
            samples.append(x)
    return np.array(samples)


# Summary all variables

import numpy as np

def problem1_rejection(n_samples=1):
    samples = []
    M = 2 * (np.exp(1) - 1) / (np.exp(1) - 2)
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1)
        u = np.random.uniform(0, 1)
        fx = 2 * x * (np.exp(x**2) - 1) / (np.exp(1) - 2)
        if u < fx / M:
            samples.append(x)
    return np.array(samples)

problem1_samples = problem1_rejection(100000)
problem1_integral = np.mean(np.sin(problem1_samples))

n = len(problem1_samples)
a = 0
b = np.sin(1)
mean = problem1_integral
delta = np.sqrt(np.log(2/0.05) * (b - a) ** 2 / (2 * n))
problem1_interval = [mean - delta, mean + delta]

def problem1_rejection_2(n_samples=1):
    samples = []
    M = 21
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1/20)
        u = np.random.uniform(0, 1)
        fx = 20 * np.exp(20 - 1/x) * (1 + 1/x)
        gx = 20
        if u < fx / (M * gx):
            samples.append(x)
    return np.array(samples)


In [ ]:
# Problem 3

# Part 1
import numpy as np
import matplotlib.pyplot as plt

def cost(y_true, y_predict_proba, threshold):
    # Predict class labels based on threshold
    y_pred = (y_predict_proba >= threshold).astype(int)
    # Calculate confusion matrix components
    TP = np.sum((y_pred == 1) & (y_true == 1))
    TN = np.sum((y_pred == 0) & (y_true == 0))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    FN = np.sum((y_pred == 0) & (y_true == 1))
    # Compute total cost
    total_cost = (TP * 100) + (FP * 120) + (FN * 600)  # TN has zero cost
    avg_cost = total_cost / len(y_true)
    return avg_cost

# Plot cost vs threshold
thresholds = np.arange(0, 1.01, 0.01)
costs = [cost(PROBLEM3_y_true_val, PROBLEM3_y_pred_proba_val, t) for t in thresholds]

plt.figure(figsize=(8,5))
plt.plot(thresholds, costs, label="Cost")
plt.xlabel("Threshold")
plt.ylabel("Average Cost")
plt.title("Average Cost vs Threshold")
plt.grid()
plt.legend()
plt.show()


# Part 2

# Find threshold that minimizes cost
min_idx = np.argmin(costs)
problem3_threshold = thresholds[min_idx]

# Get cost at optimal threshold
problem3_cost_val = costs[min_idx]

# Predicted labels at optimal threshold
problem3_y_pred_val = (PROBLEM3_y_pred_proba_val >= problem3_threshold).astype(int)

# Precision/Recall for class 1
TP = np.sum((problem3_y_pred_val == 1) & (PROBLEM3_y_true_val == 1))
FP = np.sum((problem3_y_pred_val == 1) & (PROBLEM3_y_true_val == 0))
FN = np.sum((problem3_y_pred_val == 0) & (PROBLEM3_y_true_val == 1))
TN = np.sum((problem3_y_pred_val == 0) & (PROBLEM3_y_true_val == 0))

problem3_precision_1 = TP / (TP + FP) if (TP + FP) > 0 else 0.0
problem3_recall_1 = TP / (TP + FN) if (TP + FN) > 0 else 0.0

# Precision/Recall for class 0
problem3_precision_0 = TN / (TN + FN) if (TN + FN) > 0 else 0.0
problem3_recall_0 = TN / (TN + FP) if (TN + FP) > 0 else 0.0


# Part 3

from sklearn.metrics import accuracy_score

# 0-1 loss: minimize error rate
accuracies = [accuracy_score(PROBLEM3_y_true_val, (PROBLEM3_y_pred_proba_val >= t).astype(int)) for t in thresholds]
problem3_threshold_01 = thresholds[np.argmax(accuracies)]

# Cost difference
cost_at_01 = cost(PROBLEM3_y_true_val, PROBLEM3_y_pred_proba_val, problem3_threshold_01)
problem3_cost_difference = abs(cost_at_01 - problem3_cost_val)

# Part 4

# Use threshold that minimizes cost (problem3_threshold)
test_costs = []
y_pred_test = (PROBLEM3_y_pred_proba_test >= problem3_threshold).astype(int)
# Per-sample cost
for yt, yp in zip(PROBLEM3_y_true_test, y_pred_test):
    if yp == 1 and yt == 1:
        test_costs.append(100)
    elif yp == 0 and yt == 0:
        test_costs.append(0)
    elif yp == 1 and yt == 0:
        test_costs.append(120)
    elif yp == 0 and yt == 1:
        test_costs.append(600)
test_costs = np.array(test_costs)
n = len(test_costs)
empirical_mean = np.mean(test_costs)

# Hoeffding's inequality: with probability >= 1-delta, mean is in [mean - eps, mean + eps]
# For 95% CI, delta = 0.05
from math import sqrt, log
a, b = 0, 600  # min and max possible per-sample cost
delta = 0.05
eps = (b - a) * sqrt(log(2/delta) / (2*n))
problem3_lower_bound = empirical_mean - eps
problem3_upper_bound = empirical_mean + eps





## Free text answer

Put your explanation for part 4 below this line in this **cell**. Doubleclick to enter edit mode as before.

To construct the 95% confidence interval for the average cost on the test data, we used Hoeffding’s inequality, which is valid for independent, bounded random variables. Since each transaction’s cost is between 0 and 600, and assuming the test samples are independent, Hoeffding’s inequality gives a range around the observed mean cost where the true mean cost will lie with 95% confidence. This provides a distribution-free, reliable interval for the model’s average cost performance on new, similar data.


Problem 2
# ============================================== #
# CORRECTIONS : Page 1 should replaced by Page 8 #
# ============================================== #
In this problem we have data consisting of user behavior on a website. The pages of the website are just numbers in the dataset $0,1,2,\ldots$ and each row consists of a user, a source and a destination page. This signifies that the user was on the source page and clicked a link leading them to the destination page. The goal is to improve the user experience by decreasing load time of the next page visited, as such we need a good estimate for the next site likely to be visited. We will model this using a homogeneous Markov chain, each row in the data-file then corresponds to a single realization of a transition. 

1. [3p] Load the data in the file `data/websites.csv` and construct a matrix of size `n_pages x n_pages` which is the maximum likelihood estimate of the true transition matrix for the Markov chain. Here the ordering of the states are exactly the ones in the data-file, that is page $0$ has index $0$ in the matrix.
2. [4p] A page loads in $\text{Exp}(3)$ (Exponentially distributed with mean $1/3$) seconds if not preloaded and loads with $\text{Exp}(20)$ (Exponentially distributed with mean $1/20$) seconds if preloaded and we only preload the most likely next site. Given that we start in page $8$ simulate $10000$ load times from page $8$ (that is, only a single step), store the result in the variable indicated in the cell.
Repeat the experiment but this time preload the two most likely pages and store the result in the indicated variable.
3. [3p] Compare the average (empirical) load time from part 2 with the theoretical one of no pre-loading. Does the load time improve, how did you come to this conclusion? (Explain in the free text field).
4. [4p] Calculate the stationary distribution of the Markov chain and calculate the expected load time with respect to the stationary distribution. 

# Problem 1
In this problem you will do rejection sampling from complicated distributions, you will also be using your samples to compute certain integrals, a method known as Monte Carlo integration: (Keep in mind that choosing a good sampling distribution is often key to avoid too much rejection)

1. [4p] Fill in the remaining part of the function `problem1_rejection` in order to produce samples from the below distribution using rejection sampling: (Hint: $F$ is the distribution function)

$$
    F[x] = 
    \begin{cases}
        0, & x \leq 0 \\
        \frac{e^{x^2}-x^2-1}{e-2}, & 0 < x < 1 \\
        1, & x \geq 1
    \end{cases}
$$

2. [2p] Produce 100000 samples (**use fewer if it takes too long (more than 10 sec) and you cannot find a solution**) and put the answer in `problem1_samples` from the above distribution and plot the histogram together with the true density. 
3. [2p] Use the above 100000 samples (`problem1_samples`) to approximately compute the integral

$$
    \int_0^{1} \sin(x) \frac{2(e^{x^2}-1) x}{e-2} dx
$$
and store the result in `problem1_integral`.

4. [2p] Use Hoeffdings inequality to produce a 95\% confidence interval of the integral above and store the result as a tuple in the variable `problem1_interval`

5. [4p] Fill in the remaining part of the function `problem1_rejection_2` in order to produce samples from the below distribution using rejection sampling:
$$
    F[x] = 
    \begin{cases}
        0, & x \leq 0 \\
        20xe^{20-1/x}, & 0 < x < \frac{1}{20} \\
        1, & x \geq \frac{1}{20}
    \end{cases}
$$
Hint: this is tricky because if you choose the wrong sampling distribution you reject at least 9 times out of 10. You will get points based on how long your code takes to create a certain number of samples, if you choose the correct sampling distribution you can easily create 100000 samples within 2 seconds.

# Problem 3


In this problem we are interested in fraud detection in an e-commerce system. In this problem we are given the outputs of a classifier that predicts the probabilities of fraud, your goal is to explore the threshold choice as in individual assignment 4. The costs associated with the predictions are:

* **True Positive (TP)**: Detecting fraud and blocking the transaction costs the company 100 (manual review etc.)
* **True Negative (TN)**: Allowing a legitimate transaction has no cost.
* **False Positive (FP)**: Incorrectly classifying a legitimate transaction as fraudulent costs 120 (customer dissatisfaction plus operational expenses for reversing the decision).
* **False Negative (FN)**: Missing a fraudulent transaction costs the company 600 (e.g., fraud loss plus potential reputational damage or penalties).

**The code cells contain more detailed instructions, THE FIRST CODE CELL INITIALIZES YOUR VARIABLES**

1. [3p] Complete filling the function `cost` to compute the average cost of a prediction model under a certain prediction threshold. Plot the cost as a function of the threshold (using the validation data provided in the first code cell of this problem), between 0 and 1 with 0.01 increments.
2. [2.5p] Find the threshold that minimizes the cost and calculate the cost at that threshold on the validation data. Also calculate the precision and recall at the optimal threshold on the validation data on class 1 and 0.
3. [2.5p] Repeat step 2, but this time find the best threshold to minimize the $0-1$ loss. Calculate the difference in cost between the threshold found in part 2 with the one just found in part 3.
3. [4p] Provide a confidence interval around the optimal cost (with $95\%$ confidence) applied to the test data and explain all the assumption you made.

In [ ]:
Your code for all the sub-parts of the rejection sampling problem looks **correct and well-structured**. Here’s a review and feedback for each part:

---

### **Part 1: `problem1_rejection`**

Your implementation:
```python
def problem1_rejection(n_samples=1):
    samples = []
    M = 2 * (np.exp(1) - 1) / (np.exp(1) - 2)
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1)
        u = np.random.uniform(0, 1)
        fx = 2 * x * (np.exp(x**2) - 1) / (np.exp(1) - 2)
        if u < fx / M:
            samples.append(x)
    return np.array(samples)
```
- **Correct logic**: You use uniform sampling on [0, 1], compute the target density, and accept/reject based on the ratio.  
- **Good choice of M**: You found the correct upper bound for `fx` over the support.

---

### **Part 2: Sampling and Plot**

```python
problem1_samples = problem1_rejection(100000)
# ... plotting code ...
```
- You sample 100,000 points and plot the histogram vs. the true density—**perfect**.

---

### **Part 3: Monte Carlo Integral**

```python
problem1_integral = np.mean(np.sin(problem1_samples))
```
- **Correct**: You are using the samples to estimate the expectation of `sin(x)` under the given density.

---

### **Part 4: Hoeffding Confidence Interval**

```python
n = len(problem1_samples)
a = 0
b = np.sin(1)
mean = problem1_integral
delta = np.sqrt(np.log(2/0.05) * (b - a) ** 2 / (2 * n))
problem1_interval = [mean - delta, mean + delta]
```
- **Correct use of Hoeffding's inequality** for a 95% confidence interval.
- You set `a` and `b` to the min/max values of `sin(x)` in `[0, 1]` (`sin(0)=0`, `sin(1)` is max).
- The math and code are correct.

---

### **Part 5: `problem1_rejection_2`**

```python
def problem1_rejection_2(n_samples=1):
    samples = []
    M = 21
    while len(samples) < n_samples:
        x = np.random.uniform(0, 1/20)
        u = np.random.uniform(0, 1)
        fx = 20 * np.exp(20 - 1/x) * (1 + 1/x)
        gx = 20  # Uniform on [0, 1/20]
        if u < fx / (M * gx):
            samples.append(x)
    return np.array(samples)
```
- **Efficient**: Samples from a uniform proposal, and the scaling factor `M` is reasonable.
- The proposal and acceptance ratio are correct for this target.

---

## **Final Remarks**

- **All your implementations are correct and efficient** as per the assignment.
- Your comments and variable names are clear.
- If you want, you may add a random seed before sampling for reproducibility (e.g., `np.random.seed(0)`).

---

**Summary:**  
Your code fulfills all requirements and is correct for each sub-question. You can submit it as is!

If you want the complete code in a single block for copy-paste, just let me know.